In [1]:
# Running installs
!pip install tensorflow==2.8.0
!pip install keras==2.8.0

In [2]:
# Importing Libraries
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import pandas as pd
import numpy as np
import tensorflow as tf

In [3]:
# TPU Initialization
try:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu_address)
  tf.config.experimental_connect_to_cluster(tpu)
  tf.tpu.experimental.initialize_tpu_system(tpu)
  strategy = tf.distribute.TPUStrategy(tpu)
  print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
  print("Number of accelerators: ", strategy.num_replicas_in_sync)
except ValueError:
  print('TPU failed to initialize.')

Running on TPU  ['10.61.151.146:8470']
Number of accelerators:  8


In [9]:
# Environment Variables
BATCH_SIZE = 128
REPLICAS = strategy.num_replicas_in_sync

In [5]:
class EEGNetV2(tf.keras.Model):
  def __init__(self, classes):
    super(EEGNetV2, self).__init__()

    self.classes = classes
    self.channels = 64
    self.samples = 128
    self.kernel_length = 128
    self.pool_length = 8
    self.num_filters = 8

    self.dropout1 = tf.keras.layers.Dropout(0.1)
    self.dropout2 = tf.keras.layers.Dropout(0.1)

    self.conv1 = tf.keras.layers.Conv2D(8, (1, self.kernel_length), padding = 'same', use_bias = False)
    self.bn1 = tf.keras.layers.BatchNormalization(axis = 1)
    self.DWConv1 = tf.keras.layers.DepthwiseConv2D((self.channels, 1), depth_multiplier = 2, depthwise_constraint = tf.keras.constraints.max_norm(1.), use_bias = False)
    self.bn2 = tf.keras.layers.BatchNormalization(axis = 1)
    self.AvPool1 = tf.keras.layers.AveragePooling2D((1, self.pool_length))
    self.SepConv1 = tf.keras.layers.SeparableConv2D(16, (1,16), padding='same', use_bias = False)
    self.bn3 = tf.keras.layers.BatchNormalization(axis = 1)
    self.AvPool2 = tf.keras.layers.AveragePooling2D((1, 8))
    self.flatten = tf.keras.layers.Flatten()
    self.classifier = tf.keras.layers.Dense(self.classes, activation = 'softmax')

  def call(self, inputs):
    x = self.conv1(inputs)
    x = self.bn1(x)
    x = self.DWConv1(x)
    x = self.bn2(x)
    x = tf.keras.activations.elu(x)
    x = self.AvPool1(x)
    x = self.dropout1(x)
    x = self.SepConv1(x)
    x = self.bn3(x)
    x = tf.keras.activations.elu(x)
    x = self.AvPool2(x)
    x = self.flatten(x)
    return self.classifier(x)

In [6]:
with strategy.scope():
  # Initialising the model
  model = EEGNetV2(4)

  # Optimizer, Accuracy & Loss Function
  optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
  test_loss = tf.keras.metrics.Mean(name='test_loss')
  loss_object = tf.keras.losses.CategoricalCrossentropy(reduction = tf.keras.losses.Reduction.NONE)
  train_accuracy = tf.keras.metrics.CategoricalAccuracy(name = 'train_accuracy')
  test_accuracy = tf.keras.metrics.CategoricalAccuracy(name = 'test_accuracy')

  def compute_loss(y, predictions):
    per_example_loss = loss_object(y, predictions)
    return tf.nn.compute_average_loss(per_example_loss, global_batch_size= BATCH_SIZE * REPLICAS)

  def train_step(inputs):
    x, y = inputs

    with tf.GradientTape() as tape:
      predictions = model(x)
      loss = compute_loss(y, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_accuracy.update_state(y, predictions)

    return loss

  def test_step(inputs):
    x, y = inputs

    predictions = model(x)
    loss = loss_object(y, predictions)

    test_loss.update_state(loss)
    test_accuracy.update_state(y, predictions)

  @tf.function
  def distributed_train_step(dataset_inputs):
    per_replica_losses = strategy.run(train_step,args=(dataset_inputs,))
    return strategy.reduce(tf.distribute.ReduceOp.SUM, per_replica_losses,
                           axis=None)

  @tf.function
  def distributed_test_step(dataset_inputs):
    strategy.run(test_step, args=(dataset_inputs,))

In [11]:
# Data Preperation (make sure data is of tf.data.Dataset format)
x_train  = tf.random.uniform(shape=[1000, 64, 480, 1])
y_train = tf.zeros(shape=[1000, 4])
x_test  = tf.random.uniform(shape=[128, 64, 480, 1])
y_test = tf.zeros(shape=[128, 4])
AUTO = tf.data.experimental.AUTOTUNE

def get_batched_dataset(x, y):
  dataset = tf.data.Dataset.from_tensor_slices((x, y))
  ataset = dataset.shuffle(2048)
  dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
  dataset = dataset.prefetch(AUTO)
  return dataset

def get_train_dataset():
  dataset = get_batched_dataset(x_train, y_train)
  dataset = strategy.experimental_distribute_dataset(dataset)
  return dataset

def get_test_dataset():
  dataset = get_batched_dataset(x_test, y_test)
  dataset = strategy.experimental_distribute_dataset(dataset)
  return dataset

In [12]:
EPOCHS = 1
with strategy.scope():
  for epoch in range(EPOCHS):
    total_loss = 0.0
    num_batches = 0

    for x in get_train_dataset():
      total_loss += distributed_train_step(x)
      num_batches += 1
    train_loss = total_loss / num_batches

    for x in get_test_dataset():
      distributed_test_step(x)

    template = ("Epoch {}, Loss: {:.2f}, Accuracy: {:.2f}, Test Loss: {:.2f}, "
                "Test Accuracy: {:.2f}")
    tf.print (template.format(epoch+1, train_loss,
                           train_accuracy.result()*100, test_loss.result() / strategy.num_replicas_in_sync,
                           test_accuracy.result()*100))

    test_loss.reset_states()
    train_accuracy.reset_states()
    test_accuracy.reset_states()

ValueError: ignored